# PDEfind for dataset 3 using splines approximation


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from aux_for_PDE_find import*

torch.manual_seed(0)
np.random.seed(0)

## Dataset preparation

In [2]:
path_train = "PDEfind_data/3.npz"
data_npz = np.load(path_train)

# Load the data from the .npz file
u = torch.from_numpy(data_npz['u']).type(torch.float32)  # u field
v = torch.from_numpy(data_npz['v']).type(torch.float32)  # v ield
x = torch.from_numpy(data_npz['x']).type(torch.float32)  # Spatial variable
y = torch.from_numpy(data_npz['y']).type(torch.float32)  # Spatial variable
t = torch.from_numpy(data_npz['t']).type(torch.float32)  # Temporal variable

print("u:", u.shape)
print("v:", v.shape)

u: torch.Size([256, 256, 201])
v: torch.Size([256, 256, 201])


# Build of $\Theta$

In [3]:
dt = t[0,0,1] - t[0,0,0]
dx = x[1,0,0] - x[0,0,0]
dy = y[0,1,0] - y[0,0,0]

s = 0.01
dudt = compute_time_derivative_spline(u, t, s).reshape(-1, 1)
dvdt = compute_time_derivative_spline(v, t, s).reshape(-1, 1)

# Compute space derivatives on u
dudx = compute_space_derivative_x_spline(u, x, 1, s)
dudx_2 = compute_space_derivative_x_spline(u, x, 2, s)
dudx_3 = compute_space_derivative_x_spline(u, x, 3, s)
dudy = compute_space_derivative_y_spline(u, y, 1, s)
dudy_2 = compute_space_derivative_y_spline(u, y, 2, s)
dudy_3 = compute_space_derivative_y_spline(u, y, 3, s)

# Compute space derivatives on v
dvdx = compute_space_derivative_x_spline(v, x, 1, s)
dvdx_2 = compute_space_derivative_x_spline(v, x, 2, s)
dvdx_3 = compute_space_derivative_x_spline(v, x, 3, s)
dvdy = compute_space_derivative_y_spline(v, y, 1, s)
dvdy_2 = compute_space_derivative_y_spline(v, y, 2, s)
dvdy_3 = compute_space_derivative_y_spline(v, y, 3, s)

#compute other terms
u2 = torch.pow(u, 2)
v2 = torch.pow(v, 2)
u3 = torch.pow(u, 3)
v3 = torch.pow(v, 3)

#compute mixed terms for u and v
u_times_dudx = torch.mul(u,dudx)
u_times_dudy = torch.mul(u,dudy)
u_times_dvdx = torch.mul(u,dvdx)
u_times_dvdy = torch.mul(u,dvdy)

v_times_dudx = torch.mul(v,dudx)
v_times_dudy = torch.mul(v,dudy)
v_times_dvdx = torch.mul(v,dvdx)
v_times_dvdy = torch.mul(v,dvdy)

#cross product
u_times_v = torch.mul(u,v)
u_times_v2 = torch.mul(u,v2)
u2_times_v = torch.mul(u2,v)

#compute bias
bias = torch.ones_like(u)

u = u.reshape(-1, 1)                # 0
v = v.reshape(-1, 1)                # 1
dudx = dudx.reshape(-1, 1)          # 2
dudx_2 = dudx_2.reshape(-1, 1)      # 3
dudx_3 = dudx_3.reshape(-1, 1)      # 4
dudy = dudy.reshape(-1, 1)          # 5
dudy_2 = dudy_2.reshape(-1, 1)      # 6
dudy_3 = dudy_3.reshape(-1, 1)      # 7
dvdx = dvdx.reshape(-1, 1)          # 8
dvdx_2 = dvdx_2.reshape(-1, 1)      # 9
dvdx_3 = dvdx_3.reshape(-1, 1)      # 10
dvdy = dvdy.reshape(-1, 1)          # 11
dvdy_2 = dvdy_2.reshape(-1, 1)      # 12
dvdy_3 = dvdy_3.reshape(-1, 1)      # 13
u2 = u2.reshape(-1, 1)              # 14
v2 = v2.reshape(-1, 1)              # 15
u3 = u3.reshape(-1, 1)              # 16
v3 = v3.reshape(-1, 1)              # 17
u_times_dudx = u_times_dudx.reshape(-1, 1)  # 18
u_times_dudy = u_times_dudy.reshape(-1, 1)  # 19
u_times_dvdx = u_times_dvdx.reshape(-1, 1)  # 20
u_times_dvdy = u_times_dvdy.reshape(-1, 1)  # 21
v_times_dudx = v_times_dudx.reshape(-1, 1)  # 22
v_times_dudy = v_times_dudy.reshape(-1, 1)  # 23
v_times_dvdx = v_times_dvdx.reshape(-1, 1)  # 24
v_times_dvdy = v_times_dvdy.reshape(-1, 1)  # 25
u_times_v = u_times_v.reshape(-1, 1)        # 26
u_times_v2 = u_times_v2.reshape(-1, 1)      # 27
u2_times_v = u2_times_v.reshape(-1, 1)      # 28
bias = bias.reshape(-1, 1)                  # 29

Theta = torch.cat([
    u,                # 0
    v,                # 1
    dudx,             # 2
    dudx_2,           # 3
    dudx_3,           # 4
    dudy,             # 5
    dudy_2,           # 6
    dudy_3,           # 7
    dvdx,             # 8
    dvdx_2,           # 9
    dvdx_3,           # 10
    dvdy,             # 11
    dvdy_2,           # 12
    dvdy_3,           # 13
    u2,               # 14
    v2,               # 15
    u3,               # 16
    v3,               # 17
    u_times_dudx,     # 18
    u_times_dudy,     # 19
    u_times_dvdx,     # 20
    u_times_dvdy,     # 21
    v_times_dudx,     # 22
    v_times_dudy,     # 23
    v_times_dvdx,     # 24
    v_times_dvdy,     # 25
    u_times_v,        # 26
    u_times_v2,       # 27
    u2_times_v,       # 28
    bias              # 29
], dim=1)

Theta_names = [
    "u",                # 0
    "v",                # 1
    "dudx",             # 2
    "dudx_2",           # 3
    "dudx_3",           # 4
    "dudy",             # 5
    "dudy_2",           # 6
    "dudy_3",           # 7
    "dvdx",             # 8
    "dvdx_2",           # 9
    "dvdx_3",           # 10
    "dvdy",             # 11
    "dvdy_2",           # 12
    "dvdy_3",           # 13
    "u2",               # 14
    "v2",               # 15
    "u3",               # 16
    "v3",               # 17
    "u_times_dudx",     # 18
    "u_times_dudy",     # 19
    "u_times_dvdx",     # 20
    "u_times_dvdy",     # 21
    "v_times_dudx",     # 22
    "v_times_dudy",     # 23
    "v_times_dvdx",     # 24
    "v_times_dvdy",     # 25
    "u_times_v",        # 26
    "u_times_v2",       # 27
    "u2_times_v",       # 28
    "bias"              # 29
]


## Regression and printing of the PDE for u

In [4]:
#con 4 funziona
np.random.seed(5)
sample_size = int(1e5)
random_indexes = np.random.choice(Theta.shape[0], sample_size, replace=False)
alpha=[0.1, 0.1, 0.0]
threshold=4
selected_names, regression_coef_ = myregression2(Theta[random_indexes],dudt[random_indexes],Theta_names,threshold=threshold,alpha=alpha,test_size=0.2)

Regression # 1
u  c=  0.4335963047083463
v  c=  0.25399677703526563
dudx_2  c=  0.05411800520672165
dudy_2  c=  0.05862415600868938
u3  c=  -0.4226214637519108
v3  c=  0.6995418449519094
u_times_v2  c=  -0.42592271269346926
u2_times_v  c=  0.7467509662268719
Mean Squared Error: 0.009117048512231956
###################### end of regression 1 ##############################

Regression # 2
u  c=  0.4711683148441069
v  c=  0.2906523015618657
dudx_2  c=  0.05471948383214761
dudy_2  c=  0.06151241029370771
u3  c=  -0.4626559314785647
v3  c=  0.6658143188974733
u_times_v2  c=  -0.46590201810513365
u2_times_v  c=  0.7130021459514004
Mean Squared Error: 0.009099415253740563
###################### end of regression 2 ##############################

Regression # 3
u  c=  0.4724511077320718
v  c=  0.28978125007491495
dudx_2  c=  0.054759153414186815
dudy_2  c=  0.061555760763368356
u3  c=  -0.4640301193056267
v3  c=  0.6667805333971639
u_times_v2  c=  -0.4672891201008902
u2_times_v  c=  0.71398791

In [5]:
printPDE_in_u(selected_names, regression_coef_)

dudt = 0.4724511077320718 * u +
       0.28978125007491495 * v +
       0.054759153414186815 * dudx_2 +
       0.061555760763368356 * dudy_2 +
       -0.4640301193056267 * u3 +
       0.6667805333971639 * v3 +
       -0.4672891201008902 * u_times_v2 +
       0.7139879133751321 * u2_times_v


## Regression and printing of the PDE for v

In [8]:
np.random.seed(3)
sample_size = int(1e5)
random_indexes = np.random.choice(Theta.shape[0], sample_size, replace=False)
alpha=[0.1, 0.1, 0.0]
threshold=4
selected_names, regression_coef_ = myregression2(Theta[random_indexes],dvdt[random_indexes],Theta_names,threshold=threshold,alpha=alpha, test_size=0.2)

Regression # 1
u  c=  -0.317298699375697
v  c=  0.40964624906477964
dvdx_2  c=  0.05342777619916518
dvdy_2  c=  0.05257019252615661
u3  c=  -0.6321773762637733
v3  c=  -0.3976196438563446
u_times_dudx  c=  0.05560689457497049
u_times_v2  c=  -0.6801586304784073
u2_times_v  c=  -0.4080047364731373
Mean Squared Error: 0.00851900867951945
###################### end of regression 1 ##############################

Regression # 2
u  c=  -0.327280362888719
v  c=  0.4269805601506839
dvdx_2  c=  0.05406742575433361
dvdy_2  c=  0.053776674383357474
u3  c=  -0.6238962667734465
v3  c=  -0.41607039810677093
u_times_v2  c=  -0.6715453247955411
u2_times_v  c=  -0.42640696185759974
Mean Squared Error: 0.008289037173651724
###################### end of regression 2 ##############################

Regression # 3
u  c=  -0.3211451021279569
v  c=  0.43250866578581676
dvdx_2  c=  0.0542231644247031
dvdy_2  c=  0.05395135468333002
u3  c=  -0.6306653289983557
v3  c=  -0.4220141672560772
u_times_v2  c=  -0.67

In [7]:
printPDE_in_v(selected_names, regression_coef_)

dvdt = -0.3211451021279569 * u +
       0.43250866578581676 * v +
       0.0542231644247031 * dvdx_2 +
       0.05395135468333002 * dvdy_2 +
       -0.6306653289983557 * u3 +
       -0.4220141672560772 * v3 +
       -0.6785937888304798 * u_times_v2 +
       -0.43246577254362845 * u2_times_v
